In [8]:
import numpy as np
import librosa

filename = r"D:\Temp\鋼琴.mp3"

# 載入音頻檔案
audio, sr = librosa.load(filename, sr=None)

# 執行短時傅立葉變換    
n_fft = 2048 * 8
hop_length = 512
stft_matrix = librosa.stft(audio, hop_length=hop_length, n_fft=n_fft)

# 提取频率和振幅信息
magnitude = np.abs(stft_matrix)
phase = np.angle(stft_matrix)

# 儲存每個時間窗口的最高振幅頻率和對應的振幅
frequency_data = []
amplitude_data = []
phases_data = []

print(f'每秒{sr/hop_length}偵')

for i in range(stft_matrix.shape[1]): 
    max_frequency_index = np.argmax(magnitude[:, i])  # 找出当前窗口最高振幅的索引
    max_frequency = librosa.fft_frequencies(sr=sr, n_fft=n_fft)[max_frequency_index]  # 将索引转换为频率
    max_amplitude = magnitude[max_frequency_index, i]  # 获取最高振幅
    phases = np.angle(stft_matrix)  # 获取整个 STFT 矩阵的相位信息
    max_phase = phases[max_frequency_index, i]  # 获取最高振幅对应的相位
    
    frequency_data.append(max_frequency)
    amplitude_data.append(max_amplitude)
    phases_data.append(max_phase)

    print(f"窗口{i+1}：最高震幅音频率为{max_frequency} Hz，最高震幅音的振幅为{max_amplitude}，最高震幅音的相位为{max_phase}")




每秒93.75偵
窗口1：最高震幅音频率为659.1796875 Hz，最高震幅音的振幅为0.5220011472702026，最高震幅音的相位为-0.9316731095314026
窗口2：最高震幅音频率为659.1796875 Hz，最高震幅音的振幅为0.6793308854103088，最高震幅音的相位为-0.7328850626945496
窗口3：最高震幅音频率为659.1796875 Hz，最高震幅音的振幅为0.8550989627838135，最高震幅音的相位为-0.5341570973396301
窗口4：最高震幅音频率为659.1796875 Hz，最高震幅音的振幅为1.049807071685791，最高震幅音的相位为-0.33582302927970886
窗口5：最高震幅音频率为659.1796875 Hz，最高震幅音的振幅为1.2609251737594604，最高震幅音的相位为-0.13818731904029846
窗口6：最高震幅音频率为659.1796875 Hz，最高震幅音的振幅为1.4880657196044922，最高震幅音的相位为0.05834731459617615
窗口7：最高震幅音频率为659.1796875 Hz，最高震幅音的振幅为1.7306296825408936，最高震幅音的相位为0.2521892488002777
窗口8：最高震幅音频率为659.1796875 Hz，最高震幅音的振幅为1.9837639331817627，最高震幅音的相位为0.4434531331062317
窗口9：最高震幅音频率为659.1796875 Hz，最高震幅音的振幅为2.246530055999756，最高震幅音的相位为0.6313418745994568
窗口10：最高震幅音频率为659.1796875 Hz，最高震幅音的振幅为2.51094126701355，最高震幅音的相位为0.814652681350708
窗口11：最高震幅音频率为659.1796875 Hz，最高震幅音的振幅为2.771623373031616，最高震幅音的相位为0.9947619438171387
窗口12：最高震幅音频率为659.1796875 Hz，最高震幅音的振幅为3.0249240398406982，最高震幅音的相位为1.1733213

In [2]:
import math
import numpy as np

def freq_to_note(freq):

    A4_freq = 440.0
    semitone_ratio = 2 ** (1/12)
    num_semitones = 12 * math.log2(int(freq) / A4_freq)
    num_semitones_rounded = round(num_semitones)

    note_names = ["A", "A#", "B", "C", "C#", "D", "D#", "E", "F", "F#", "G", "G#"]
    
    note_index = num_semitones_rounded % 12
    octave = 4 + num_semitones_rounded // 12
    
    return note_names[note_index] + str(octave)


note_to_semitone = {
    'C': 0,
    'C#': 1, 'Db': 1,
    'D': 2,
    'D#': 3, 'Eb': 3,
    'E': 4,
    'F': 5,
    'F#': 6, 'Gb': 6,
    'G': 7,
    'G#': 8, 'Ab': 8,
    'A': 9,
    'A#': 10, 'Bb': 10,
    'B': 11
}

def note_to_freq(note):
    # 提取音名和八度
    if len(note) == 3:  # like C#4 or Db4
        note_name = note[:2]
        octave = int(note[2])
    else:  # like C4
        note_name = note[0]
        octave = int(note[1])
    
    # 計算相對於C0的半音數
    semitone_offset = note_to_semitone[note_name]
    
    # 計算MIDI音高數
    midi_number = semitone_offset + (octave + 1) * 12
    
    return 440.0 * (2 ** ((midi_number - 69) / 12))


notes = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
note_list = [note + str(octave) for octave in range(1, 9) for note in notes]

def get_note(value):
    try:
        # 找到指定音符的索引
        index = note_list.index(value)
        # 如果索引为0，说明是第一个音符，没有前一个音符
        if index == 0:
            return None
        else:
            # 返回前一个音符
            return [note_list[index - 1],note_list[index + 1]]
    except ValueError:
        # 如果指定音符不在列表中，返回None
        return None

def change_pitch(replacement_value, value):
    try:
        for idx in range(len(frequency_data)):  # 假设范围是100到200
            freq = frequency_data[idx]
            if freq > note_to_freq(get_note(value)[0]) and freq < note_to_freq(get_note(value)[1]):
                print(f'把頻率{freq}的音改為{note_to_freq(replacement_value)}')
                frequency_data[idx] = note_to_freq(replacement_value)
                #print(frequency[idx])
    except ValueError:
        return None

replacement_value = 'E3'
value = 'C4'
print(change_pitch(replacement_value, value))

None


In [10]:
import numpy as np
import librosa
import soundfile as sf

# 讀取音頻信息
sr = 44100 
n_fft = 2048 * 8
hop_length = 512



# 初始化一個空的STFT矩陣，用於存儲處理後的頻率和振幅數據
reconstructed_stft = np.zeros((n_fft//2 + 1, len(frequency_data)), dtype=np.complex64)

for i in range(len(frequency_data)):
    max_frequency = frequency_data[i]
    max_amplitude = amplitude_data[i]
    max_phase = phases_data[i]

    # 找到與max_frequency最接近的頻率索引
    freq_idx = np.argmin(np.abs(librosa.fft_frequencies(sr=sr, n_fft=n_fft) - max_frequency))
    
    # 重建STFT矩陣
    reconstructed_stft[freq_idx, i] = max_amplitude * np.exp(1j * max_phase)

# 進行逆STFT
reconstructed_audio = librosa.istft(reconstructed_stft, hop_length=hop_length)

# 如果需要播放音頻，可以使用以下代碼（需要安裝IPython.display）
from IPython.display import Audio
Audio(data=reconstructed_audio, rate=sr)